<a href="https://colab.research.google.com/github/kingslyt/huggingface_experiments/blob/master/datasets_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2022-01-30 20:23:52--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2022-01-30 20:23:52--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘SQuAD_it-train.json.gz’

SQuAD_it-train.json 100%[===================>]   7.37M  --.-KB/s    in 0.1s    

2022-01-30 20:23:52 (61.2 MB/s) - ‘SQuAD_it-train.json.gz’ saved [7725286/7725286]

--202

In [2]:
!gzip -dkv SQuAD_it-*.json.gz


SQuAD_it-test.json.gz:	 87.5% -- replaced with SQuAD_it-test.json
SQuAD_it-train.json.gz:	 82.3% -- replaced with SQuAD_it-train.json


In [4]:
!ls -ltrh

total 58M
drwxr-xr-x 1 root root 4.0K Jan  7 14:33 sample_data
-rw-r--r-- 1 root root 7.4M Jan 30 20:23 SQuAD_it-train.json.gz
-rw-r--r-- 1 root root  42M Jan 30 20:23 SQuAD_it-train.json
-rw-r--r-- 1 root root 1.1M Jan 30 20:23 SQuAD_it-test.json.gz
-rw-r--r-- 1 root root 8.0M Jan 30 20:23 SQuAD_it-test.json


In [5]:
!head SQuAD_it-train.json

{
    "data": [
        {
            "title": "Terremoto del Sichuan del 2008",
            "paragraphs": [
                {
                    "context": "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
                    "qas": [
                        {
                            "id": "56cdca7862d2951400fa6826",


In [7]:
!pip install datasets

     |████████████████████████████████| 312 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 243 kB 56.3 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 133 kB 48.2 MB/s 
     |████████████████████████████████| 144 kB 50.9 MB/s 
     |████████████████████████████████| 271 kB 54.7 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 


In [8]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

Using custom data configuration default-79ef2a63cdd180fc


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-79ef2a63cdd180fc/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

In [17]:
dataset = squad_it_dataset["train"]


# 90% train, 10% test + validation
train_test_dataset = dataset.train_test_split(test_size=0.1, seed=42, shuffle=True)
train_test_dataset

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-79ef2a63cdd180fc/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-4b60d62d479f12cb.arrow and /root/.cache/huggingface/datasets/json/default-79ef2a63cdd180fc/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-38b54c94b68f224d.arrow


DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 397
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 45
    })
})

In [20]:
from datasets import DatasetDict

# Split the 10% test + valid in half test, half valid
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5, seed=42, shuffle=True)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/json/default-79ef2a63cdd180fc/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-6fb4806624c4046b.arrow and /root/.cache/huggingface/datasets/json/default-79ef2a63cdd180fc/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-cc4843d2b8177dae.arrow
